# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Lee  ->  K. Lee  |  ['K. Lee']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 76 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2402.11188


extracting tarball to tmp_2402.11188...

 done.
Retrieving document from  https://arxiv.org/e-print/2402.12063


extracting tarball to tmp_2402.12063... done.
Retrieving document from  https://arxiv.org/e-print/2402.12084


extracting tarball to tmp_2402.12084...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2402.12084/FRB190520B_Swift_FAST.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.11188-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.11188) | **AGN properties of ~1 million member galaxies of galaxy groups and  clusters at z < 1.4 based on the Subaru Hyper Suprime-Cam survey**  |
|| Y. Toba, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2024-02-20*|
|*Comments*| *25 pages, 24 figures, and 3 tables, accepted for publication in ApJ. A value-added CAMIRA member galaxy catalog and the best-fit SED for each member galaxy will be available as FITS or machine-readable tables*|
|**Abstract**| Herein, we present the statistical properties of active galactic nuclei (AGNs) for approximately 1 million member galaxies of galaxy groups and clusters, with 0.1 $<$ cluster redshift ($z_{\rm cl}$) $<$ 1.4, selected using Subaru Hyper Suprime-Cam, the so-called CAMIRA clusters. In this research, we focused on the AGN power fraction ($f_{\rm AGN}$), which is defined as the proportion of the contribution of AGNs to the total infrared (IR) luminosity, $L_{\rm IR}$ (AGN)/$L_{\rm IR}$, and examined how $f_{\rm AGN}$ depends on (i) $z_{\rm cl}$ and (ii) the distance from the cluster center. We compiled multiwavelength data using the ultraviolet--mid-IR range. Moreover, we performed spectral energy distribution fits to determine $f_{\rm AGN}$ using the CIGALE code with the SKIRTOR AGN model. We found that (i) the value of $f_{\rm AGN}$ in the CAMIRA clusters is positively correlated with $z_{\rm cl}$, with the correlation slope being steeper than that for field galaxies, and (ii) $f_{\rm AGN}$ exhibits a high value at the cluster outskirts. These results indicate that the emergence of AGN population depends on the redshift and environment and that galaxy groups and clusters at high redshifts are important in AGN evolution. Additionally, we demonstrated that cluster--cluster mergers may enhance AGN activity at the outskirts of particularly massive galaxy clusters. Our findings are consistent with a related study on the CAMIRA clusters that was based on the AGN number fraction. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.12063-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.12063) | **To test $R_{NLRs}~-~L_{O3}$ relation for narrow emission line regions of  AGN through low redshift Type-2 AGN in SDSS**  |
|| <mark>X. Zhang</mark> |
|*Appeared on*| *2024-02-20*|
|*Comments*| *10 pages, 10 figures, Accepted to be published in MNRAS*|
|**Abstract**| Sizes of narrow emission line regions (NLRs) of AGN could be estimated by [O~{\sc iii}] line luminosity $L_{O3}$ through the known $R_{NLRs}-L_{O3}$ empirical relations. Unfortunately, it is not convenient to test the $R_{NLRs}-L_{O3}$ empirical relations through structure properties of spatially resolved NLRs of large samples of AGN. In this manuscript, a method is proposed to test the $R_{NLRs}-L_{O3}^{\sim0.25}$ empirical relations for AGN NLRs through SDSS Type-2 AGN having few orientation effects on NLRs sizes expected by AGN unified model, after considering sizes $R_{fib}$ of SDSS fiber covered regions. Comparing $R_{fib}$ and $R_{NLRs}$ estimated by $L_{O3}$, Type-2 AGN with $R_{fib}>R_{NLRs}$ (Sample-II) and with $R_{fib}<R_{NLRs}$ (Sample-I) should have different physical properties of NLRs. Accepted electron density gradients in AGN NLRs, statistically higher electron densities (traced by lower flux ratio $R_{S2}$ of [S~{\sc ii}]$\lambda6717$\AA~ to [S~{\sc ii}]$\lambda6731$\AA) could be expected for the Type-2 AGN in the Sample-I. Then, through the collected 1062 SDSS Type-2 AGN in the Sample-I and 3658 SDSS Type-2 AGN in the Sample-II, statistically lower $R_{S2}$ for the Type-2 AGN in the Sample-I can be confirmed with confidence level higher than 5$\sigma$, even after considering necessary effects. Therefore, the results in this manuscript can provide strong clues to support that the reported $R_{NLRs}~\propto~L_{O3}^{0.25}$ empirical relation is preferred to estimate NLRs sizes of SDSS AGN through SDSS fiber spectroscopic results, and also to support the commonly expected electron density gradients in AGN NLRs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.12084-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.12084) | **Simultaneous multi-wavelength observations of the repeating fast radio  burst FRB 20190520B with Swift and FAST**  |
|| Z. Yan, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-02-20*|
|*Comments*| *12 pages, 7 figures, submitted to ApJ*|
|**Abstract**| Fast radio bursts (FRBs) are bright, millisecond-duration radio bursts of cosmic origin. There have been several dozen FRBs found to repeat. Among them, those precisely localized provide the best opportunity to probe their multi-wavelength counterparts, local environment, and host galaxy that would reveal their origins. Here we report our X-ray, ultraviolet (UV) and optical observations with the $Swift$ satellite that were performed simultaneously in the radio band with the Five-hundred-meter Aperture Spherical radio Telescope (FAST) observations of the repeating FRB 20190520B, aiming at detection of possible multi-wavelength bursts in association with radio bursts and multi-wavelength counterpart of the persistent radio source (PRS). While a total of 10 radio bursts were detected by FAST at the same time of $Swift$ observations, we detected neither X-ray, UV or optical bursts in accompany of the radio bursts, nor persistent multi-wavelength counterpart of the PRS. We obtained the energy upper limits ($3\sigma$) on any multi-wavelength bursts as $5.03 \times 10^{47}$ erg in the hard X-ray band (15-150 keV), $7.98 \times 10^{45}$ erg in the soft X-ray band (0.3-10 keV), and $4.51 \times 10^{44}$ erg in the U band, respectively. The energy ratio between soft X-ray (0.3-10 keV) and radio emission of the bursts is constrained as $<6\times10^{7}$, and the ratio between optical (U band) and radio as $<1.19\times10^{6}$. The 3$\sigma$ luminosity upper limits at the position of PRS are 1.04$\times10^{47}$ (15-150 keV), 8.81$\times10^{42}$ (0.3-10 keV), 9.26$\times10^{42}$ (UVW1), and 2.54$\times10^{42}$ erg s$^{-1}$ (U), respectively. We show that the PRS is much more radio loud than representative pulsar wind nebulae, supernova remnants, extended jet of Galactic X-ray binaries and ultraluminous X-ray sources, suggestive of boosted radio emission of the PRS. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

372  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
